## **演示0804：模型性能分析**

### **提出问题：**
当我们使用一批训练数据，训练出一个假设函数(模型)后，一般会有几个疑问：  
* 这个模型的效果(性能)如何呢，或者借助该模型预测出来的结果，可信度有多高呢？
* 如果预测结果不理想，那么可以从哪些方面去改进呢？

### **用于模型验证和评价的数据**
假设手头有1000条原始数据，可按照6:2:2的比例进行分割
* 60%的训练样本(Training Examples)：用于训练判别式。要通过调整$\lambda$, learning rate等，分别计算$J_{train}(\theta)$，从而得到若干个候选的判别函数
* 20%的验证样本(Validation Examples)：对前面的每个候选判别式，分别计算$J_{val}(\theta)$，取最小的一个作为最终的判别式
* 20%的测试样本(Test Examples)：用于计算前述选出的判别式的$J_{test}(\theta)$的预测正确率。这个正确率可以作为模型的泛化效果，也就是说在处理新数据时的可信度。  
* $J_{val}(\theta)$，$J_{test}(\theta)$的计算公式与$J_{train}(\theta)$相同，只是将数据集换成对应的验证数据集或测试数据集

### **模型性能尺度**
假设某疾病诊断Logistic Regression模型$h_\theta(x)$，如果$y$=1，代表发现了疾病，$y$=0代表没有疾病。对于给定的test data，我们预测错误率只有1%（99%的正确率），那么可以认为这个模型很好吗？  
假设在真实情况下，只有0.5%的几率有疾病；那么我们完全可以设计一个极简单的算法（永远预测$y$=0），这样的话，也只有0.5%的错误率（比我们之前的$h_\theta(x)$还要好得多）。但是，这个极简单的算法就会更好吗？
因此，除了使用我们通常认为的错误率/正确率来衡量一个模型判别式外，还应有其它更为重要的验算指标。  
* 预测和实际结果统计表(假设仅考虑2-Classes分类的情形)  
![](../images/080401.png)  
 * Actual Class表示数据集中实际的结果；Predicted Class则表示通过Hypothesis预测的结果
 * True：表示实际结果与预测结果一致(预测正确)；False：表示实际结果和预测结果不一致(预测错误)
 * Positive：表示预测结果为1；Negative表示预测结果为0
 * 上述各中情况将分别简写为：TP, FP, TN, FN。建议可以这样理解记忆：针对一条测试数据
   * TP：模型预测结果为P(Positive, 1)，而且预测正确（该测试数据实际结果也为Positive/1)
   * FP：模型预测结果为P(Positive, 1)，但是预测错误（该测试数据实际结果应为Negtive/0)
   * TN：模型预测结果为N(Negtive, 0)，而且预测正确（该测试数据实际结果也为Negtive/0)
   * FN：模型预测结果为N(Negtive, 0)，但是预测错误（该测试数据实际结果应为Postive/1)\
* 尺度计算
 * 正确率：$ Accuracy=\frac{\#TP+\#TN}{\#TP+\#FP+\#TN+\#FN} $  
体现了：预测正确的样本数占总样本数的比重
 * 精度：$ Precision= \frac{\#TP}{\#TP+\#FP} $  
体现了：对于所有预测为1的样本中，实际真正为1的样本所占的比例。它反映**“误报”**程度：精度越高，误报越小
 * 查全率：$ Recall = \frac{\#TP}{\#TP+\#FN} $  
体现了：对于所有实际为1的样本中，预测也为1的样本所占的比例。它反映**“漏报”**程度：查全率越高，漏报越少
 * $F1 Score = \dfrac{2 PR}{P+R} $  
其中，P为Precision, R为Recall  
在针对Validation Data进行对比时，应选择F1 Score最大的那个判别式来作为最优解  
对于前面的例子，如果某个算法简单预测所有$y=0$，那么其查全率为0，F1 Score=0，因此模型性能最差

* 在Precision和Recall中取平衡  
假设某Logistic Regression算法，其判别式输出：$ 0 \le h_\theta(x) \le 1$。设计某个阈值$K$，使得当$h_\theta(x) \ge K$时，预测$y = 1$；当$h_\theta(x) \lt K$时，预测$y=0$  
 * 通常，我们设计K为0.5。也就是说，当可能性超过0.5时就做出Positive的预测结果
 * 如果设置K=0.7，意味着预测更加保守，只有高可信度才会被预测为1。 因此误报可能性降低，Precision提升；漏报可能性增加，Recall降低。
 * 如果设置K=0.3，意味着预测趋于大胆，低可信度也会被预测为1。因此误报可能性增加，Precision降低；漏报可能性减少，Recall上升。
 * 大致可以得出结论：Precisioin和Recall是有矛盾的，它们很难同时都达到最高。因此需要取折中(F1 Score)

### **训练样本数量对模型性能的影响**
有时候模型的预测性能很差，有可能是模型算法的问题，也有可能是样本的问题。其中，样本数量不足可能是影响模型性能的一个重要因素。  
以训练样本数量$m$(training example)为横坐标，以模型最终计算出来的$J(\theta)$为纵坐标作图，该图称为学习曲线(Learning Curves)
* 对于High Variane（过拟合），其$J_{train}(\theta)$一般很小，且随着$m$的增加缓慢的增加  
![](../images/080402.png)  
* 对于High Bias（欠拟合），其$J_{train}(\theta)$刚开始很小，但是随着$m$的增加，会迅速增加；$J_{val}(\theta)$通常很大，而且随着$m$的增加缓慢的减少，最终二者比较接近且值都较大。在这种情况下，增加更多的训练数据没有任何帮助，而是要考虑改进算法或选取更多的Feature  
![](../images/080403.png)  
* 当$J_{train}(\theta)$趋于稳定时的$m$数，可作为比较合适的训练样本数量